In [ ]:
#Даниил Дорожкин
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from rapidfuzz.distance import Levenshtein
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm

tqdm.pandas()

employees = pd.read_parquet("employees.parquet")
orcs = pd.read_parquet("orcs.parquet")
print(f"Employees: {len(employees)}, orcs: {len(orcs)}")

# приведение к привычному виду
def normalize_fio(s):
    if pd.isna(s):
        return ""
    return str(s).lower().replace(" ", "").replace(".", "").replace(",", "").replace("*", "").replace("`", "").replace("-", "")
#без устраненния очевидных опечаток tf iff работал хуже, а метрика левенштейна не меняется

def normalize_date(d):
    if pd.isna(d):
        return ""
    try:
        return pd.to_datetime(d).strftime("%Y%m%d")
    except:
        return ""

print("нормализация")
for df in [employees, orcs]:
    df["fio"] = df.progress_apply(
        lambda r: normalize_fio(r["surname"]) +
                  normalize_fio(r["name"]) +
                  normalize_fio(r["fathername"]),
        axis=1
    )
    df["norm_birth"] = df["birthdate"].apply(normalize_date)

#точные совпадения по паспопт и инн, допускаем, что тут уже опеаток нет, по нескоьким проверкам с поискам сотрудника по орочьему паспорту 
#найдено аж 75%, значит опечаток действительно нет или их очень мало, на всякий случай
#можно учесть в левенштейне, чтобы избежать пропусков со слегка отличающимися серией или номером

#фамилии и имена содержали явные опечатки, явно не только в 25% данных
print("Поиск точных совпадений")
strict_mask = (
    employees["passport"].isin(orcs["passport"]) |
    employees["inn"].isin(orcs["inn"])
)
strict_matches = employees[strict_mask].index.values
print(f"Точные совпадения: {len(strict_matches)}")

orc_candidates = orcs[
    ~(orcs["passport"].isin(employees["passport"]) |
      orcs["inn"].isin(employees["inn"]))
].copy()

emp_candidates = employees[~strict_mask].copy()

print(f"Потенциально остается: Employees={len(emp_candidates)}, orcs={len(orc_candidates)}")

#Векторизация

# LSH / datasketch MinHash сильно хуже на коротких строках
# splink сильно медленнее
print("Векторизация")
vec = TfidfVectorizer(analyzer="char", ngram_range=(2, 4)) #analyzer="char" — устойчивость к опечаткам
#ngram_range(2,4) — учитывает маленькие кусочки текста (би-три-квадро граммы)
X_emp = vec.fit_transform(emp_candidates["fio"] + emp_candidates["norm_birth"])
X_orc = vec.transform(orc_candidates["fio"] + orc_candidates["norm_birth"])

# ---------- NN----------
K = 200 #выбираем не одного, так как малое косинусное расстояние между векторами не гарантирует фактическое совпадение
#строк, которые их задали, с большой вероятностью будет настоящий двойник из другого списка
#без левенштейна будет много ложноположительных

#это делитель 37245, чтобы не создавать громадную матрицу, вообще время работы программы отличаться не будет просто матрица расширится в бок
#но вот потом на этапе проверки 
print(f"Поиск ближайших соседей (top {K})...")
nn = NearestNeighbors(n_neighbors=K, metric="cosine", n_jobs=-1)
nn.fit(X_emp)

batch_size = 1600 #TF IDF большой, ограничиваем нагрузку, тут параметры подлобраны на счет 0.9462, увеличивая батч и чсило соседей, добился 0.9576 
#прога при этом работала 40 минут, но фильтрация по левенштейну затянулась
#в тоерии можно сдеелать без разбиения на батчи и без ограничения по количеству соседей, но тогда фильтрация по левенштейну будет длиться более суток (проверено)
orc_knn = []

#X_emp — TF-IDF матрица сотрудников
#X_orc — TF-IDF матрица орков

#kneighbors находит топ-K ближайших
#idx массив индексов сотрудников-кандидатов
#Если batch = 3 орка и K=5, то матрица idx размером (3 × 5)
#(batch_size × K)


for i in tqdm(range(0, X_orc.shape[0], batch_size)):
    _, idx = nn.kneighbors(X_orc[i:i+batch_size]) #перебираем каждого орка батчами, возврааем матрицу ближайших сотрудников
    orc_knn.extend([emp_candidates.index.values[ix] for ix in idx])
    #emp_candidates.index.values[ix] — переводим индекс в оригинальный индекс таблицы сотрудников (до фильтра)

orc_candidates["knn_idxs"] = orc_knn
#у каждого орка есть поле knn_idxs, содержащее по K ближайших сотрудников

# Левенштейн
print("Левенштейн")
selected = []

for _, row in tqdm(orc_candidates.iterrows(), total=len(orc_candidates)):
    best_score = 1.0
    best_emp_idx = None

    for emp_idx in row["knn_idxs"]:
        emp_row = employees.loc[emp_idx]

        if emp_row["gender"] != row["gender"]:
            continue

        scores = [] #нормализованное расстояние, 0 - идеально, порог 0.25 допускаем для условного кирилл киллер в усреднении с датой и другими данными
        scores.append(Levenshtein.normalized_distance(row["fio"], emp_row["fio"]))

        if row["norm_birth"] and emp_row["norm_birth"]:
            scores.append(Levenshtein.normalized_distance(row["norm_birth"], emp_row["norm_birth"]))

#в целом это необязаительно, просто и так не объединяли в одну строку ФИО+ДР, но идею "а вдруг" объяснил 
        if pd.notna(row["passport"]) and pd.notna(emp_row["passport"]):
            scores.append(Levenshtein.normalized_distance(str(row["passport"]), str(emp_row["passport"])))

        if pd.notna(row["inn"]) and pd.notna(emp_row["inn"]):
            scores.append(Levenshtein.normalized_distance(str(row["inn"]), str(emp_row["inn"])))

        avg_score = np.mean(scores) #страховка от шумов и пропусков

        if avg_score < best_score:
            best_score = avg_score
            best_emp_idx = emp_idx

            #среди ближайших соседей считаем разные метрики и берем действительно максимально похожего best_emp_idx = emp_idx

    if best_emp_idx is not None and best_score < 0.25: #покрывает две опечатки или перестановки в имени/ фамилии
        #по наблюдениям, 0.3 - хуже, а промежуточные не пробовал 
        selected.append(best_emp_idx)

selected = np.unique(np.array(selected, np.uint64))

#объединяем новых и очевидных
final_orcs = np.unique(np.concatenate([strict_matches, selected]))
print(f"Всего орков среди сотрудников надено: {len(final_orcs)}")


#сохраняем
res = pd.DataFrame({
    "orig_index": final_orcs.astype(np.uint64),
}).reset_index(names="id")

res.to_parquet("submission.parquet", index=False)


Employees: 2011759, orcs: 47633
нормализация


100%|██████████| 47633/47633 [00:00<00:00, 69886.14it/s]


Поиск точных совпадений
Точные совпадения: 10388
Потенциально остается: Employees=2001371, orcs=37245
Векторизация
Поиск ближайших соседей (top 200)...


100%|██████████| 24/24 [26:37<00:00, 66.55s/it]


Левенштейн


100%|██████████| 37245/37245 [02:15<00:00, 275.78it/s]

Всего орков среди сотрудников надено: 13814
